# 内容一：在MNIST数据集上构建网络进行分类

## 1. 实验前导

In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import torch.utils.data as tud
import numpy as np

## 2. 准备数据

### 学会使用Dataloader来加载数据
Dataloader能够帮我们打乱数据集，拿到batch数据 \
为了使用Dataloader，需要定义以下三个function
- \__init__: 模型初始化
- \__len__: 返回整个数据集有多少item
- \__getitem__: 根据给定的index返回一个item

调用Dataloader之前还要先定义dataset

In [2]:
# Pytorch帮助我们预先加载了一些常用的数据集
# 如果使用这些数据集，会相对容易的进行数据加载
# 例如：常用的Mnist数据集
mnist_train_data = datasets.MNIST("./data",train=True,download=True,
                                 transform = transforms.Compose([
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean=(0.13066062,),std=(0.30810776,))
                                 ]))
batch_size = 64
train_dataloader = tud.DataLoader(mnist_train_data,batch_size = batch_size,shuffle=True) # 将dataset转换为iterator
mnist_test_data = datasets.MNIST("./data",train=False,download=True,
                                 transform = transforms.Compose([
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean=(0.13066062,),std=(0.30810776,))
                                 ]))
test_dataloader = tud.DataLoader(mnist_test_data,batch_size = batch_size)

## 3. 配置网络

### (1) 定义网络
- 继承 nn.Module
- 初始化函数
- forward 函数
- 其余可以根据模型需要定义相关的函数

In [3]:
# 定义一个简单的基于ConvNet的简单神经网络
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__() # the input is 1*28*28
        self.conv1 = nn.Conv2d(1,20,5,1) # (28-5)/1+1=24, 20*24*24
        self.conv2 = nn.Conv2d(20,50,5,1) # 12-5+1=8
        self.fc1 = nn.Linear(4*4*50,500)
        self.fc2 = nn.Linear(500,10)
    def forward(self,x):
        x = F.relu(self.conv1(x)) # 20 * 24 * 24
        x = F.max_pool2d(x,2,2) # 20 * 12 * 12
        x = F.relu(self.conv2(x)) # 50 * 8 * 8
        x = F.max_pool2d(x,2,2) # 50 * 4 * 4
        x = x.view(-1,4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x #F.log_softmax(x,dim=1)
model = Net()

### (2) 定义损失函数

In [4]:
loss_fn = nn.CrossEntropyLoss(reduction='mean')

### (3) 定义优化算法

In [5]:
lr = 0.01
momentum = 0.5
optimizer = optim.SGD(model.parameters(),lr=lr,momentum=momentum)

## 4. 训练网络

- 模型一般需要训练若干个epoch
- 每个epoch我们把所有数据分成若干个batch
- 把每个batch的输入和输出都包装成cuda Tensor
- forward pass
- 计算loss
- 清空gradient
- backward pass
- 更新模型参数
- 每隔一定的iteration输出loss，以及在验证集上做模型的评估

In [6]:
def train(model,train_dataloader,loss_fn,optimizer,epoch):
    model.train()
    for idx, (data, label) in enumerate(train_dataloader):
        output = model(data)
        loss = loss_fn(output,label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if idx % 100 == 0:
            print("Train Epoch: {}, iteration: {}, loss: {}".format(
                epoch,idx,loss.item()))  
    return model

## 5. 模型评估

In [14]:
def test(model,test_dataloader,loss_fn):
    model.eval()
    total_loss = 0.
    correct = 0.
    with torch.no_grad():
        for idx, (data,target) in enumerate(test_dataloader):
            output = model(data) # batch_size * 10        
            loss = loss_fn(output,target)*output.size(0)
            pred = output.argmax(dim=1)
            total_loss += loss
            correct += pred.eq(target).sum()
    total_loss /= len(test_dataloader.dataset)
    acc = 100.*correct/len(test_dataloader.dataset)
    print("Test Loss:{}, Accuracy:{}".format(total_loss,acc))
    return acc

In [12]:
num_epochs = 2
for epoch in range(num_epochs):
    model = train(model,train_dataloader,loss_fn,optimizer,epoch)
    test(model,test_dataloader,loss_fn)

Train Epoch: 0, iteration: 0, loss: 0.009488344192504883
Train Epoch: 0, iteration: 100, loss: 0.1327199935913086
Train Epoch: 0, iteration: 200, loss: 0.01545263547450304
Train Epoch: 0, iteration: 300, loss: 0.012227410450577736
Train Epoch: 0, iteration: 400, loss: 0.010789387859404087
Train Epoch: 0, iteration: 500, loss: 0.03559090942144394
Train Epoch: 0, iteration: 600, loss: 0.20145973563194275
Train Epoch: 0, iteration: 700, loss: 0.03271903470158577
Train Epoch: 0, iteration: 800, loss: 0.02920989692211151
Train Epoch: 0, iteration: 900, loss: 0.027539599686861038
Test Loss:0.038897547870874405, Accuracy:98
Train Epoch: 1, iteration: 0, loss: 0.041452355682849884
Train Epoch: 1, iteration: 100, loss: 0.010027386248111725
Train Epoch: 1, iteration: 200, loss: 0.004974566865712404
Train Epoch: 1, iteration: 300, loss: 0.05770661681890488
Train Epoch: 1, iteration: 400, loss: 0.15365464985370636
Train Epoch: 1, iteration: 500, loss: 0.001344119431450963
Train Epoch: 1, iteration

## 6. 模型存储

In [15]:
#torch.save(model.state_dict(),"mnist_cnn.pth")
# num_epochs = 2
best_valid_acc = 0.
for epoch in range(num_epochs):
    train(model,train_dataloader,loss_fn,optimizer,epoch)
    acc = test(model,test_dataloader,loss_fn)
    if acc > best_valid_acc:
        best_valid_acc = acc
        torch.save(model.state_dict(),"best_mnist_cnn.pth")

Train Epoch: 0, iteration: 0, loss: 0.019819634035229683
Train Epoch: 0, iteration: 100, loss: 0.005775895901024342
Train Epoch: 0, iteration: 200, loss: 0.0707644447684288
Train Epoch: 0, iteration: 300, loss: 0.06397535651922226
Train Epoch: 0, iteration: 400, loss: 0.014788621105253696
Train Epoch: 0, iteration: 500, loss: 0.006537220440804958
Train Epoch: 0, iteration: 600, loss: 0.038363415747880936
Train Epoch: 0, iteration: 700, loss: 0.006373587995767593
Train Epoch: 0, iteration: 800, loss: 0.003357541048899293
Train Epoch: 0, iteration: 900, loss: 0.0035391380079090595
Test Loss:0.038350194692611694, Accuracy:98
Train Epoch: 1, iteration: 0, loss: 0.054631706327199936
Train Epoch: 1, iteration: 100, loss: 0.09225575625896454
Train Epoch: 1, iteration: 200, loss: 0.0015655563911423087
Train Epoch: 1, iteration: 300, loss: 0.04000664874911308
Train Epoch: 1, iteration: 400, loss: 0.02154170535504818
Train Epoch: 1, iteration: 500, loss: 0.009217644110321999
Train Epoch: 1, iter

### Load模型

In [16]:
test_model = Net()
test_model.load_state_dict(torch.load("mnist_cnn.pth"))
test(model,test_dataloader,loss_fn)

Test Loss:0.028637487441301346, Accuracy:99


tensor(99)

### For FashionMNIST

In [22]:
batch_size = 32
train_dataloader = tud.DataLoader(
    datasets.FashionMNIST("./fashion_mnist_data",train=True,download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize(mean=(0.2860402,),std=(0.3530239,))
                   ])),
    batch_size=batch_size,
    shuffle=True) # 将dataset转换为iterator
test_dataloader = tud.DataLoader(
    datasets.FashionMNIST("./fashion_mnist_data",train=False,download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize(mean=(0.2860402,),std=(0.3530239,))
                   ])),
    batch_size=batch_size) # 将dataset转换为iterator
lr = 0.01
momentum = 0.5
model = Net()
optimizer = optim.SGD(model.parameters(),lr=lr,momentum=momentum)
num_epochs = 2

for epoch in range(num_epochs):
    train(model,train_dataloader,loss_fn,optimizer,epoch)
    test(model,test_dataloader,loss_fn)
    
torch.save(model.state_dict(),"fashion_mnist_cnn.pth")

Train Epoch: 0, iteration: 0, loss: 2.3000686168670654
Train Epoch: 0, iteration: 100, loss: 0.855004608631134
Train Epoch: 0, iteration: 200, loss: 0.9181270599365234
Train Epoch: 0, iteration: 300, loss: 0.9267852902412415
Train Epoch: 0, iteration: 400, loss: 0.661331295967102
Train Epoch: 0, iteration: 500, loss: 0.8637953400611877
Train Epoch: 0, iteration: 600, loss: 0.5660000443458557
Train Epoch: 0, iteration: 700, loss: 0.6191807985305786
Train Epoch: 0, iteration: 800, loss: 0.8863906860351562
Train Epoch: 0, iteration: 900, loss: 0.3977414071559906
Train Epoch: 0, iteration: 1000, loss: 0.33772027492523193
Train Epoch: 0, iteration: 1100, loss: 0.3567613363265991
Train Epoch: 0, iteration: 1200, loss: 0.4134741425514221
Train Epoch: 0, iteration: 1300, loss: 0.42777499556541443
Train Epoch: 0, iteration: 1400, loss: 0.30109351873397827
Train Epoch: 0, iteration: 1500, loss: 0.5608471035957336
Train Epoch: 0, iteration: 1600, loss: 0.4938139319419861
Train Epoch: 0, iteration

# 内容二：CNN模型的迁移学习

- 很多时候当我们训练一个新的图像分类任务，我们不会完全从一个随机的模型开始训练，而是利用预训练的模型来加速训练的过程。我们经常使用在ImageNet上的预训练模型
- 有两种方法做迁移学习
    - finetuning：从一个预训练模型开始，改变一些模型的架构，然后继续训练整个模型的参数；
    - feature extraction：不改变预训练模型的参数，只更新我们改变过的部分模型参数。（当成特征提取器来使用）

## 1. 实验前导

In [23]:
import os
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import models, datasets, transforms
import torch.utils.data as tud
import numpy as np

## 2. 准备数据

数据：使用hymenoptera_data数据集 \
数据集包括两类图片，bees和ants。这些数据都被处理成了可以使用ImageFolder来读取的格式。我们只需要把data_dir设置成数据的根目录，然后把model_name设置成我们想要使用的预训练模型

In [24]:
# format
data_dir = "./data/hymenoptera_data"
model_name = "resnet18"
num_class = 2
#feature_extract = True
input_size = 224

读入数据: 把数据预处理成相应的格式

In [25]:
# progress
all_imgs = datasets.ImageFolder(os.path.join(data_dir,"train"),
                                transform=transforms.Compose([
                                    transforms.RandomResizedCrop(input_size),
                                    transforms.RandomHorizontalFlip(),
                                    transforms.ToTensor(),                                    
                                ]))
loader = tud.DataLoader(all_imgs,batch_size=batch_size,shuffle=True)

In [26]:
all_imgs[20][0]

tensor([[[0.6784, 0.6667, 0.6745,  ..., 0.4784, 0.4667, 0.4471],
         [0.6667, 0.6588, 0.6627,  ..., 0.4745, 0.4706, 0.4667],
         [0.6902, 0.6863, 0.6863,  ..., 0.4863, 0.4863, 0.4784],
         ...,
         [0.4549, 0.4902, 0.4784,  ..., 0.3843, 0.4706, 0.4510],
         [0.5490, 0.4784, 0.5137,  ..., 0.4392, 0.4706, 0.4667],
         [0.6431, 0.5451, 0.5569,  ..., 0.4941, 0.3961, 0.4039]],

        [[0.6902, 0.6941, 0.6902,  ..., 0.4706, 0.4627, 0.4510],
         [0.6941, 0.6941, 0.6980,  ..., 0.4784, 0.4706, 0.4588],
         [0.7020, 0.6980, 0.6902,  ..., 0.4784, 0.4745, 0.4667],
         ...,
         [0.4667, 0.4980, 0.4824,  ..., 0.3686, 0.4745, 0.4431],
         [0.5569, 0.4863, 0.5176,  ..., 0.4235, 0.4627, 0.4784],
         [0.6902, 0.5843, 0.5961,  ..., 0.5137, 0.3922, 0.3961]],

        [[0.7373, 0.7294, 0.7216,  ..., 0.4941, 0.4902, 0.4824],
         [0.7255, 0.7216, 0.7176,  ..., 0.5059, 0.5020, 0.4863],
         [0.7373, 0.7255, 0.7216,  ..., 0.5059, 0.5098, 0.

In [27]:
# format
batch_size = 32
train_imgs = datasets.ImageFolder(os.path.join(data_dir,"train"),
                                transform=transforms.Compose([
                                    transforms.RandomResizedCrop(input_size),
                                    transforms.RandomHorizontalFlip(),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.485, 0.456, 0.406],[0.229,0.224,0.225])
                                ]))
train_dataloader = tud.DataLoader(train_imgs,batch_size=batch_size,shuffle=True)
test_imgs = datasets.ImageFolder(os.path.join(data_dir,"val"),
                                transform=transforms.Compose([
                                    transforms.Resize(input_size),  
                                    transforms.CenterCrop(input_size),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.485, 0.456, 0.406],[0.229,0.224,0.225])
                                ]))
test_dataloader = tud.DataLoader(test_imgs,batch_size=batch_size)

## 3. 配置网络

### (1) 定义网络

In [28]:
# format
def initialize_model(model_name,num_class,use_pretrained=True,feature_extract=True):
    if model_name == "resnet18":
        model_ft = models.resnet18(pretrained=use_pretrained)
        if feature_extract: # do not update the parameters
            for param in model_ft.parameters():
                param.requires_grad = False
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs,num_class)        
    else:
        print("model not implemented")
        return None
    return model_ft
model_ft = initialize_model("resnet18",2,use_pretrained=False,feature_extract=False)

In [29]:
print(model_ft.layer1[0].conv1.weight.requires_grad)
print(model_ft.fc.weight.requires_grad)

True
True


### (2) 定义损失函数

In [30]:
loss_fn = nn.CrossEntropyLoss()

### (3) 定义优化算法

In [31]:
lr = 0.01
momentum = 0.5
optimizer = optim.SGD(model_ft.parameters(),lr=lr,momentum=momentum)

## 4. 训练网络

In [32]:
def train_model(model,train_dataloader,loss_fn,optimizer,epoch):
    model.train()
    total_loss = 0.
    total_corrects = 0.
    for idx, (inputs, labels) in enumerate(train_dataloader):
        outputs = model(inputs)
        loss = loss_fn(outputs,labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        preds = outputs.argmax(dim=1)
        total_loss += loss.item() * inputs.size(0)
        total_corrects += torch.sum(preds.eq(labels))
    epoch_loss = total_loss / len(train_dataloader.dataset)
    epoch_accuracy = total_corrects / len(train_dataloader.dataset)
    print("Epoch:{}, Training Loss:{}, Traning Acc:{}".format(epoch,epoch_loss,epoch_accuracy))  
    #return model        

## 5. 模型评估

In [33]:
def test_model(model,test_dataloader,loss_fn):
    model.eval()
    total_loss = 0.
    total_corrects = 0.
    with torch.no_grad():
        for idx, (inputs, labels) in enumerate(test_dataloader):
            outputs = model(inputs)
            loss = loss_fn(outputs,labels)
            preds = outputs.argmax(dim=1)
            total_loss += loss.item() * inputs.size(0)
            total_corrects += torch.sum(preds.eq(labels))
    epoch_loss = total_loss / len(test_dataloader.dataset)
    epoch_accuracy = total_corrects / len(test_dataloader.dataset)
    print("acc type:", epoch_accuracy)
    print("Test Loss:{}, Test Acc:{}".format(epoch_loss,epoch_accuracy))  
    return epoch_accuracy 

In [ ]:
num_epochs = 5
for epoch in range(num_epochs):
    train_model(model_ft,train_dataloader,loss_fn,optimizer,epoch)
    acc = test_model(model_ft,test_dataloader,loss_fn)

Epoch:0, Training Loss:0.8485996078272335, Traning Acc:0
Test Loss:0.722400501857396, Test Acc:0
Epoch:1, Training Loss:0.7474724112964067, Traning Acc:0
Test Loss:0.6469219225684023, Test Acc:0
Epoch:2, Training Loss:0.6163499804793812, Traning Acc:0
Test Loss:0.5612572902947469, Test Acc:0
